In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [2]:
@lru_cache(maxsize=None)
def load_bank():
    # https://archive.ics.uci.edu/ml/datasets/Bank+Marketing
    df=pd.read_csv('data/bank.csv', sep=';',header=0)
    # Basic data cleanup to get rid of bad values and turn Y into 0-1 val
    df=df.dropna(axis=1, how='all')
    df=df.dropna(axis=0, how='any')
    cat_cols=[]
    i = 0 
    for eachcol in df.dtypes:
        if eachcol.name=="object":
            cat_cols.append(df.columns[i])
        i=i+1
    # Convert the string values into integers, and give each value its own column, hot encode
    df=pd.get_dummies(df,columns=cat_cols)
    df.head()
    X=df.iloc[:,0:-2]
    y=df['y_yes']
    # Use sklearn to split up the dataset
    return X, y


In [3]:
@lru_cache(maxsize=None)
def load_heart():
    df = pd.read_csv("data/heart.csv")
    X = df.iloc[:,0:-1]
    y = df.iloc[:,-1]
    return X, y

In [4]:
X, y = load_heart()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # Could Add a RandomState number to set the rand kernel

# Decision Tree 

In [14]:
classifiers = [
    (DecisionTreeClassifier(max_depth=5), "Decision Tree"),
    (MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(50, 20), random_state=1), "Neural Network"),
    (AdaBoostClassifier(n_estimators=100, random_state=0), "Boost"),
    (svm.SVC(), "SVM"),
    (KNeighborsClassifier(3), "KNN")
]
for clf in classifiers:
    model = clf[0].fit(x_train, y_train)
    accuracy = model.score(x_test, y_test)
    print(clf[1])
    print("\taccuracy: {0:.3f}%".format(accuracy))


Decision Tree
	accuracy: 0.672%
Neural Network
	accuracy: 0.787%
Boost
	accuracy: 0.672%
SVM
	accuracy: 0.656%
KNN
	accuracy: 0.607%


/home/builder/ml-env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
